In [2]:
from IPython.display import display, clear_output
import ipywidgets as widgets

# Função para formatar valores como moeda BRL
def formatar_brl(valor):
    s = f"{valor:,.2f}"
    s = s.replace(',', 'X').replace('.', ',').replace('X', '.')
    return s

# Função para formatar taxa com até 23 casas decimais
def formatar_taxa(valor):
    return f"{valor:,.23f}".replace('.', ',')

# Saída principal
main_output = widgets.Output()

# Botões principais
compra_button = widgets.Button(description='COMPRA', button_style='primary')
venda_button = widgets.Button(description='VENDA', button_style='danger')
valor_em_reais_button = widgets.Button(description='VALOR EM REAIS', button_style='success')
concomitante_button = widgets.Button(description='CONCOMITANTE', button_style='warning')
calcular_button = widgets.Button(description='CALCULAR', button_style='primary')
limpar_button = widgets.Button(description='LIMPAR', button_style='danger')
voltar_button = widgets.Button(description='VOLTAR', button_style='info')

# Campos PONTA VENDA (para CONCOMITANTE)
ponta_venda_valor = widgets.FloatText(description='Valor Incidente:', step=0.01, min=0.01)
ponta_venda_ir = widgets.Dropdown(
    options=[('ISENTO', 0.0), ('15% (CREDOR)', 0.15), ('17,64706% (DEVEDOR)', 0.1764706)],
    description='I.R:'
)
ponta_venda_iof = widgets.Dropdown(
    options=[('ISENTO', 1.0), ('1,10%', 1.011), ('3,5%', 1.035)],
    description='IOF:'
)
ponta_venda_tarifa = widgets.Dropdown(
    options=[('ISENTO', 0.0), ('R$ 250,00', 250.0)],
    description='Tarifa:'
)
ir_warning_label_concomitante = widgets.HTML(value="", layout={'visibility': 'hidden', 'margin': '0px 0px 0px 10px'})

# Campos PONTA COMPRA (para CONCOMITANTE, sem I.R., IOF fixo)
ponta_compra_iof = widgets.Dropdown(
    options=[('0,38%', 0.0038)],
    description='IOF Compra:'
)
ponta_compra_tarifa = widgets.Dropdown(
    options=[('ISENTO', 0.0), ('R$ 250,00', 250.0)],
    description='Tarifa Compra R$:'
)

# Labels de saída para CONCOMITANTE
saida_concomitante = {
    "resumo_valores_operacao": widgets.HTML(),
    "resumo_ponta_venda": widgets.HTML(),
    "resumo_ponta_compra": widgets.HTML(),
    "resumo_final": widgets.HTML(),
}

# Função para exibir CONCOMITANTE
def mostrar_concomitante(b):
    with main_output:
        clear_output(wait=True)
        display(widgets.VBox([
            widgets.HTML("<h4>🔧 Módulo CONCOMITANTE</h4>"),
            widgets.HTML("<b>PONTA VENDA:</b>"),
            ponta_venda_valor,
            widgets.HBox([ponta_venda_ir, ir_warning_label_concomitante]),
            ponta_venda_iof,
            ponta_venda_tarifa,
            widgets.HTML("<b>PONTA COMPRA:</b>"),
            ponta_compra_iof,
            ponta_compra_tarifa,
            widgets.HBox([calcular_button, limpar_button, voltar_button]),
            widgets.HTML("<hr>"),
            saida_concomitante["resumo_valores_operacao"],
            widgets.HTML("<hr>"),
            saida_concomitante["resumo_ponta_venda"],
            widgets.HTML("<hr>"),
            saida_concomitante["resumo_ponta_compra"],
            widgets.HTML("<hr>"),
            saida_concomitante["resumo_final"]
        ]))

# Função para atualizar o aviso de I.R. no CONCOMITANTE
def update_ir_warning_concomitante(change):
    if change['new'] == 0.1764706:  # 17,64706% (DEVEDOR)
        ir_warning_label_concomitante.value = (
            "<span style='color: red; font-size: 12px;'>"
            "Neste caso o I.R será debitado à parte. Para debitar o valor do I.R. do principal, selecione alíquota 15% (CREDOR)"
            "</span>"
        )
        ir_warning_label_concomitante.layout.visibility = 'visible'
    else:
        ir_warning_label_concomitante.value = ""
        ir_warning_label_concomitante.layout.visibility = 'hidden'

ponta_venda_ir.observe(update_ir_warning_concomitante, names='value')

# Função de cálculo para CONCOMITANTE
def atualizar_valores_concomitante(b):
    try:
        venda_valor = ponta_venda_valor.value
        venda_ir_pct = ponta_venda_ir.value
        venda_iof_pct = ponta_venda_iof.value
        tarifa_venda = ponta_venda_tarifa.value

        # Calcula o valor do boleto VENDA
        if venda_ir_pct == 0.15:
            valor_boleto_venda = venda_valor * (1 - 0.15)
        elif venda_ir_pct == 0.1764706:
            valor_boleto_venda = venda_valor / (1 + 0.1764706)
        else:
            valor_boleto_venda = venda_valor

        # Calcula IOF da ponta VENDA
        valor_iof_venda = (valor_boleto_venda * venda_iof_pct) - valor_boleto_venda if venda_iof_pct != 1.0 else 0.0

        # IOF fixo da ponta COMPRA (0,38%)
        compra_iof_pct = ponta_compra_iof.value
        tarifa_compra = ponta_compra_tarifa.value

        # Calcula Valor total Ponta Compra (GROSS UP) como Valor IOF Venda + 0.38145%
        valor_total_ponta_compra = valor_iof_venda * (1 + 0.0038145)

        # O valor da ponta COMPRA é igual ao Valor total Ponta Compra (GROSS UP)
        compra_valor = valor_total_ponta_compra

        # Calcula IOF da ponta COMPRA como Valor total Ponta Compra (GROSS UP) * 0.38%
        valor_iof_compra = valor_total_ponta_compra * 0.0038

        # NET IOF COMPRA é igual ao Valor IOF Venda
        net_iof_compra = valor_iof_venda

        # Calcula o valor total CREDITADO
        valor_total_creditado = compra_valor - valor_iof_compra - tarifa_compra

        # Calcula o valor total DEBITADO
        valor_ir_venda = venda_valor * venda_ir_pct if venda_ir_pct != 0.0 else 0.0
        valor_total_debitado = valor_boleto_venda + valor_ir_venda + valor_iof_venda + tarifa_venda

        # Calcula o valor total da operação
        valor_total_operacao = valor_boleto_venda + valor_ir_venda + valor_iof_venda + tarifa_venda

        # Calcula o VALOR REAL A SER ENVIADO
        valor_real_enviado = valor_boleto_venda - tarifa_venda - valor_iof_compra - tarifa_compra

        # Valor Incidente Aprovado em Pré Análise é igual ao valor inserido pelo usuário
        valor_incidente_aprovado = venda_valor

        # Atualiza as saídas
        saida_concomitante["resumo_valores_operacao"].value = (
            f"<h4>VALORES DA OPERAÇÃO</h4>"
            f"<b>Valor Boleto VENDA:</b> R$ {formatar_brl(valor_boleto_venda)}<br>"
            f"<b>Valor Boleto COMPRA:</b> R$ {formatar_brl(compra_valor)}"
        )
        saida_concomitante["resumo_ponta_venda"].value = (
            f"<h4>RESUMO DA OPERAÇÃO - PONTA VENDA</h4>"
            f"<b>Valor do Imposto de Renda:</b> "
            f"{'ISENTO' if valor_ir_venda == 0 else f'R$ {formatar_brl(valor_ir_venda)}'}<br>"
            f"<b>Valor IOF Venda:</b> R$ {formatar_brl(valor_iof_venda)}<br>"
            f"<b>Valor Tarifa Venda:</b> R$ {formatar_brl(tarifa_venda)}<br>"
            f"<b>Valor total DEBITADO:</b> R$ {formatar_brl(valor_total_debitado)}"
        )
        saida_concomitante["resumo_ponta_compra"].value = (
            f"<h4>RESUMO DA OPERAÇÃO - PONTA COMPRA</h4>"
            f"<b>Valor total Ponta Compra (GROSS UP):</b> R$ {formatar_brl(valor_total_ponta_compra)}<br>"
            f"<b>NET IOF COMPRA:</b> R$ {formatar_brl(net_iof_compra)}<br>"
            f"<b>Valor IOF Ponta Compra (0,38%):</b> R$ {formatar_brl(valor_iof_compra)}<br>"
            f"<b>Valor Tarifa Compra:</b> R$ {formatar_brl(tarifa_compra)}<br>"
            f"<b>Valor total CREDITADO:</b> R$ {formatar_brl(valor_total_creditado)}"
        )
        saida_concomitante["resumo_final"].value = (
            f"<h4>FINAL</h4>"
            f"<b>VALOR INCIDENTE APROVADO EM PRÉ ANÁLISE:</b> R$ {formatar_brl(valor_incidente_aprovado)}<br>"
            f"<b>VALOR TOTAL DA OPERAÇÃO:</b> R$ {formatar_brl(valor_total_operacao)}<br>"
            f"<b>VALOR REAL A SER ENVIADO:</b> R$ {formatar_brl(valor_real_enviado)}"
        )
    except Exception as e:
        for label in saida_concomitante.values():
            label.value = f"<b>Erro no cálculo:</b> {str(e)}"

# Função LIMPAR para CONCOMITANTE
def limpar_campos_concomitante(b):
    ponta_venda_valor.value = 0.0
    ponta_venda_ir.value = 0.0
    ponta_venda_iof.value = 1.0
    ponta_venda_tarifa.value = 0.0
    ponta_compra_iof.value = 0.0038
    ponta_compra_tarifa.value = 0.0
    ir_warning_label_concomitante.value = ""
    ir_warning_label_concomitante.layout.visibility = 'hidden'
    for label in saida_concomitante.values():
        label.value = ""

# Função para voltar ao menu inicial
def voltar_menu(b):
    with main_output:
        clear_output(wait=True)
        intro_text = widgets.HTML("<h3>Bem-vindo à Calculadora de Câmbio do Gabrielino</h3><p>O que você deseja calcular?</p>")
        display(widgets.VBox([
            intro_text,
            widgets.HBox([compra_button, venda_button, valor_em_reais_button, concomitante_button])
        ]))

# Função para exibir VALOR EM REAIS
def mostrar_valor_em_reais(b):
    valor_boleto_calculado = None
    with main_output:
        clear_output(wait=True)
        valor_input = widgets.FloatText(value=0.00, description='Valor (R$):', step=0.01, min=0.01, max=999999999.99)
        iof_dropdown = widgets.Dropdown(options=[('ISENTO', 1.0), ('3,5%', 1.035), ('1,10%', 1.011)], value=1.035, description='IOF:')
        ir_dropdown = widgets.Dropdown(options=[('ISENTO', 0.0), ('15% (CREDOR)', 0.15), ('17,64706% (DEVEDOR)', 0.1764706)], value=0.0, description='I.R:')
        tarifa_dropdown = widgets.Dropdown(options=[('ISENTO', 0.0), ('R$ 250,00', 250.0)], value=0.0, description='Tarifa:')
        ir_warning_label = widgets.HTML(value="", layout={'visibility': 'hidden', 'margin': '0px 0px 0px 10px'})
        calcular_button_local = widgets.Button(description='CALCULAR VALOR DO BOLETO', button_style='success')
        limpar_button_local = widgets.Button(description='LIMPAR', button_style='danger')
        abrir_taxa_button = widgets.Button(description='ABRIR A TAXA', button_style='info')
        output_local = widgets.Output()

        def update_ir_warning(change):
            if change['new'] == 0.1764706:  # 17,64706% (DEVEDOR)
                ir_warning_label.value = (
                    "<span style='color: red; font-size: 12px;'>"
                    "Neste caso o I.R será debitado à parte. Para debitar o valor do I.R. do principal, selecione alíquota 15% (CREDOR)"
                    "</span>"
                )
                ir_warning_label.layout.visibility = 'visible'
            else:
                ir_warning_label.value = ""
                ir_warning_label.layout.visibility = 'hidden'

        ir_dropdown.observe(update_ir_warning, names='value')

        def calcular_boleto(b):
            nonlocal valor_boleto_calculado
            with output_local:
                clear_output()
                try:
                    valor_original = valor_input.value
                    tarifa = tarifa_dropdown.value
                    iof_fator = iof_dropdown.value
                    ir_pct = ir_dropdown.value

                    if valor_original < 0.01 or valor_original > 999999999.99:
                        print("❌ Valor fora do intervalo permitido.")
                        return

                    # Calcular o valor do boleto
                    resultado = valor_original - tarifa
                    if ir_pct == 0.15:  # 15% (CREDOR)
                        resultado -= resultado * 0.15
                    elif ir_pct == 0.1764706 or ir_pct == 0.0:  # 17,64706% (DEVEDOR) ou ISENTO
                        resultado -= 0.0

                    # Aplicar IOF para o boleto
                    if iof_fator == 1.035:  # 3,5%
                        resultado_final = resultado / 1.035
                        valor_iof = resultado_final * 0.035
                    elif iof_fator == 1.011:  # 1,10%
                        resultado_final = resultado / 1.011
                        valor_iof = resultado_final * 0.011
                    else:  # ISENTO
                        resultado_final = resultado / 1.0
                        valor_iof = 0.0

                    # Calcular I.R. para exibição no Resumo da Operação
                    if ir_pct == 0.15 or ir_pct == 0.1764706:  # 15% (CREDOR) ou 17,64706% (DEVEDOR)
                        valor_ir = (resultado_final + valor_iof) * 0.1764706
                    else:  # ISENTO
                        valor_ir = 0.0

                    # Calcular Valor Total da Operação (Valor do boleto + IOF + I.R. + Tarifa)
                    valor_total_operacao = resultado_final + valor_iof + valor_ir + tarifa

                    # Armazenar o valor do boleto
                    valor_boleto_calculado = resultado_final

                    # Construir o Resumo da Operação
                    memoria = []
                    memoria.append(f"Valor inicial: R$ {formatar_brl(valor_original)}")
                    memoria.append(f"Valor total da operação: R$ {formatar_brl(valor_total_operacao)}")
                    memoria.append(f"I.R.: {'ISENTO' if valor_ir == 0.0 else f'R$ {formatar_brl(valor_ir)}'}")
                    memoria.append(f"IOF: {'ISENTO' if valor_iof == 0 else f'R$ {formatar_brl(valor_iof)}'}")
                    memoria.append(f"Tarifa: {'ISENTO' if tarifa == 0.0 else f'R$ {formatar_brl(tarifa)}'}")

                    print(f"✅ Valor do boleto: R$ {formatar_brl(resultado_final)}")
                    print("\n🧮 Resumo da Operação:")
                    for item in memoria:
                        print(item)

                except Exception as e:
                    print(f"Erro: {e}")

        def limpar_campos_valor_em_reais(b):
            nonlocal valor_boleto_calculado
            valor_input.value = 0.0
            iof_dropdown.value = 1.035
            ir_dropdown.value = 0.0
            tarifa_dropdown.value = 0.0
            ir_warning_label.value = ""
            ir_warning_label.layout.visibility = 'hidden'
            valor_boleto_calculado = None
            with output_local:
                clear_output()

        def abrir_taxa(b):
            with main_output:
                clear_output(wait=True)
                valor_em_reais_input = widgets.FloatText(
                    value=round(valor_boleto_calculado, 2) if valor_boleto_calculado is not None else 0.00,
                    description='VALOR R$:',
                    step=0.01,
                    min=0.01
                )
                valor_em_me_input = widgets.FloatText(
                    value=0.0,
                    description='VALOR M.E.:',
                    step=0.01,
                    min=0.01
                )
                valor_reais_warning_label = widgets.HTML(
                    value="<span style='color: blue; font-size: 12px;'>INSIRA AQUI O VALOR EM REAIS CONTRATADO</span>",
                    layout={'margin': '0px 0px 0px 10px'}
                )
                valor_me_warning_label = widgets.HTML(
                    value="<span style='color: blue; font-size: 12px;'>INSIRA AQUI O VALOR EM MOEDA ESTRANGEIRA</span>",
                    layout={'margin': '0px 0px 0px 10px'}
                )
                abrir_taxa_calcular_button = widgets.Button(description='ABRIR TAXA', button_style='primary')
                voltar_valor_em_reais_button = widgets.Button(description='VOLTAR', button_style='info')
                output_taxa = widgets.Output()

                def calcular_taxa(b):
                    with output_taxa:
                        clear_output()
                        try:
                            valor_reais = valor_em_reais_input.value
                            valor_me = valor_em_me_input.value
                            if valor_me == 0:
                                print("❌ M.E. não pode ser zero.")
                                return
                            taxa = valor_reais / valor_me
                            print(f"TAXA ABERTA R$ {formatar_taxa(taxa)}")
                        except Exception as e:
                            print(f"Erro: {e}")

                abrir_taxa_calcular_button.on_click(calcular_taxa)
                voltar_valor_em_reais_button.on_click(mostrar_valor_em_reais)

                display(widgets.VBox([
                    widgets.HTML("<h4>🔧 Calcular Taxa</h4>"),
                    widgets.HBox([valor_em_reais_input, valor_reais_warning_label]),
                    widgets.HBox([valor_em_me_input, valor_me_warning_label]),
                    widgets.HBox([abrir_taxa_calcular_button, voltar_valor_em_reais_button]),
                    output_taxa
                ]))

        calcular_button_local.on_click(calcular_boleto)
        limpar_button_local.on_click(limpar_campos_valor_em_reais)
        abrir_taxa_button.on_click(abrir_taxa)

        display(widgets.VBox([
            widgets.HTML("<h4>🔧 Módulo VALOR EM REAIS</h4>"),
            widgets.HTML("<p>Aqui vamos calcular o valor do boleto quando temos um valor exato em reais a ser remetido ao exterior. Quando precisamos garantir que nenhum centavo a mais seja debitado da conta do cliente.</p>"),
            valor_input,
            iof_dropdown,
            widgets.HBox([ir_dropdown, ir_warning_label]),
            tarifa_dropdown,
            widgets.HBox([calcular_button_local, limpar_button_local, abrir_taxa_button, voltar_button]),
            output_local
        ]))

# Função para exibir MÓDULO VENDA
def mostrar_venda(b):
    with main_output:
        clear_output(wait=True)
        valor_venda = widgets.FloatText(value=0.00, description='Valor em R$:', step=0.01, min=0.01)
        iof_venda = widgets.Dropdown(options=[('ISENTO', 1.0), ('3,5%', 1.035), ('1,10%', 1.011)], value=1.035, description='IOF:')  # 3,5% padrão
        ir_venda = widgets.Dropdown(options=[('ISENTO', 0.0), ('15% (CREDOR)', 0.15), ('17,64706% (DEVEDOR)', 0.1764706)], value=0.0, description='I.R:')
        tarifa_venda = widgets.Dropdown(options=[('ISENTO', 0.0), ('R$ 250,00', 250.0)], value=250.0, description='Tarifa:')
        calcular_venda_button = widgets.Button(description='CALCULAR', button_style='primary')
        limpar_venda_button = widgets.Button(description='LIMPAR', button_style='danger')
        output_venda = widgets.HTML()

        def calcular_venda(b):
            try:
                valor_input = valor_venda.value
                iof_fator = iof_venda.value
                ir_pct = ir_venda.value
                tarifa = tarifa_venda.value

                # Ajusta o valor da operação se I.R. for por conta do CREDOR (15%)
                if ir_pct == 0.15:
                    valor = valor_input * (1 - 0.15)
                else:
                    valor = valor_input

                # Calcula IOF
                valor_iof = (valor * iof_fator) - valor if iof_fator != 1.0 else 0.0

                # Calcula I.R.
                valor_ir = valor * ir_pct if ir_pct != 0.0 else 0.0

                # Calcula Total da Operação
                total_operacao = valor + valor_iof + valor_ir + tarifa

                # Atualiza saída
                output_venda.value = (
                    f"<h4>RESUMO OPERAÇÃO VENDA</h4>"
                    f"<b>Valor em Reais:</b> R$ {formatar_brl(valor)}<br>"
                    f"<b>IOF:</b> {'ISENTO' if valor_iof == 0 else f'R$ {formatar_brl(valor_iof)}'}<br>"
                    f"<b>I.R.:</b> {'ISENTO' if valor_ir == 0 else f'R$ {formatar_brl(valor_ir)}'}<br>"
                    f"<b>Tarifa:</b> R$ {formatar_brl(tarifa)}<br>"
                    f"<b>Total da Operação:</b> R$ {formatar_brl(total_operacao)}"
                )
            except Exception as e:
                output_venda.value = f"<b>Erro no cálculo:</b> {str(e)}"

        def limpar_venda(b):
            valor_venda.value = 0.0
            iof_venda.value = 1.035  # Retorna para 3,5%
            ir_venda.value = 0.0
            tarifa_venda.value = 250.0
            output_venda.value = ""

        calcular_venda_button.on_click(calcular_venda)
        limpar_venda_button.on_click(limpar_venda)

        display(widgets.VBox([
            widgets.HTML("<h4>🔧 Módulo VENDA (ENVIO)</h4><p>AQUI VAMOS CALCULAR TODOS OS CUSTOS DE ENVIO DE REMESSAS DE CÂMBIO</p>"),
            valor_venda, iof_venda, ir_venda, tarifa_venda,
            widgets.HBox([calcular_venda_button, limpar_venda_button, voltar_button]),
            widgets.HTML("<hr>"),
            output_venda
        ]))

# Função para exibir MÓDULO COMPRA
def mostrar_compra(b):
    with main_output:
        clear_output(wait=True)
        valor_compra = widgets.FloatText(value=0.00, description='Valor em R$:', step=0.01, min=0.01)
        iof_compra = widgets.Dropdown(options=[('ISENTO', 0.0), ('0,38%', 0.0038)], value=0.0, description='IOF:')
        tarifa_compra = widgets.Dropdown(options=[('ISENTO', 0.0), ('R$ 250,00', 250.0)], value=250.0, description='Tarifa:')
        calcular_compra_button = widgets.Button(description='CALCULAR', button_style='primary')
        limpar_compra_button = widgets.Button(description='LIMPAR', button_style='danger')
        output_compra = widgets.HTML()

        def calcular_compra(b):
            try:
                valor = valor_compra.value
                iof_pct = iof_compra.value
                tarifa = tarifa_compra.value

                # Calcula IOF
                valor_iof = valor * iof_pct if iof_pct != 0.0 else 0.0

                # Calcula Valor Total a Ser Creditado
                total_creditado = valor - valor_iof - tarifa

                # Atualiza saída
                output_compra.value = (
                    f"<h4>RESUMO OPERAÇÃO COMPRA</h4>"
                    f"<b>Valor em Reais:</b> R$ {formatar_brl(valor)}<br>"
                    f"<b>IOF:</b> {'ISENTO' if valor_iof == 0 else f'R$ {formatar_brl(valor_iof)}'}<br>"
                    f"<b>Tarifa:</b> R$ {formatar_brl(tarifa)}<br>"
                    f"<b>Valor Total a Ser Creditado:</b> R$ {formatar_brl(total_creditado)}"
                )
            except Exception as e:
                output_compra.value = f"<b>Erro no cálculo:</b> {str(e)}"

        def limpar_compra(b):
            valor_compra.value = 0.0
            iof_compra.value = 0.0
            tarifa_compra.value = 250.0
            output_compra.value = ""

        calcular_compra_button.on_click(calcular_compra)
        limpar_compra_button.on_click(limpar_compra)

        display(widgets.VBox([
            widgets.HTML("<h4>🔧 Módulo COMPRA (RECEBIMENTO)</h4><p>AQUI VAMOS CALCULAR OS CUSTOS DE RECEBIMENTO DE REMESSAS DE CÂMBIO</p>"),
            valor_compra, iof_compra, tarifa_compra,
            widgets.HBox([calcular_compra_button, limpar_compra_button, voltar_button]),
            widgets.HTML("<hr>"),
            output_compra
        ]))

# Ligações dos botões
calcular_button.on_click(atualizar_valores_concomitante)
limpar_button.on_click(limpar_campos_concomitante)
voltar_button.on_click(voltar_menu)
concomitante_button.on_click(mostrar_concomitante)
valor_em_reais_button.on_click(mostrar_valor_em_reais)
compra_button.on_click(mostrar_compra)
venda_button.on_click(mostrar_venda)

# Exibição inicial
with main_output:
    clear_output(wait=True)
    intro_text = widgets.HTML("<h3>Bem-vindo à Calculadora de Câmbio do Gabrielino</h3><p>O que você deseja calcular?</p>")
    display(widgets.VBox([intro_text, widgets.HBox([compra_button, venda_button, valor_em_reais_button, concomitante_button])]))

# Exibir a saída principal
display(main_output)

Output()